In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from collections import Counter
import ast  

Move integron.filter.tsv file to another dir

In [ ]:
path='Path/to/test_bacant_result/'
output='Path/to/integron_result/'

if not os.path.exists(output):
    os.makedirs(output)

all_folder=os.listdir(path)
for i in all_folder:
    if os.path.exists(path+i+"/integron.filter.tsv"):
        shutil.copyfile(path+i+"/integron.filter.tsv", output+i+'.tsv')

Move transposon.filter.tsv file to another dir

In [ ]:
path='Path/to/test_bacant_result/'
output='Path/to/transposon_result/'

if not os.path.exists(output):
    os.makedirs(output)

all_folder=os.listdir(path)
for i in all_folder:
    if os.path.exists(path+i+"/transposon.filter.tsv"):
        if os.path.getsize(path+i+"/transposon.filter.tsv") > 162:
            shutil.copyfile(path+i+"/transposon.filter.tsv", output+i+'.tsv')

Motifi transposon file

In [ ]:
########   Transposon Motifi  #######
transposon_path='Path/to/transposon_result/'
transposon_out='Path/to/Motifi_transposon/'
transposon_list=os.listdir(transposon_path)

if not os.path.exists(transposon_out):
    os.makedirs(transposon_out)

for i in transposon_list:
    new_column=[]
    file=pd.read_csv(transposon_path+i,sep='\t')

    file = pd.DataFrame({
        "Contig": file["QUERY"],
        "Transposon_Type": file["TRANSPOSON TYPE|ACCESSION"],
        "Start": file["QUERY START"],
        "End": file["QUERY END"]})
    
    file["Location"] = file["Start"].astype(str).str.cat(file["End"].astype(str), sep="-")
    file=file.drop("Start", axis=1)
    file=file.drop("End",axis=1)
#####For Transposon Motifi, code bellow is needed********
    file_contig=[]
    temp_file_contig=file['Contig'].to_list()
    for ii in temp_file_contig:
        ii=ii.split('~')[0]
        file_contig.append(ii)
    file['Contig']=file_contig


    file_transposon=[]
    temp_file_transposon=file['Transposon_Type'].to_list()
    for ii in temp_file_transposon:
        ii=ii.split('|')[0]
        file_transposon.append(ii)
    
    file["Transposon_Type"]=file_transposon
    file.to_csv(transposon_out+i,index=False)

Motifi integron file

In [ ]:
########   Integron Motifi  #######
integron_path='Path/to/integron_result/'
integron_out='Path/to/Motifi_integron/'
integron_list=os.listdir(integron_path)

if not os.path.exists(integron_out):
    os.makedirs(integron_out)

for i in integron_list:
    new_column=[]
    file=pd.read_csv(integron_path+i,sep='\t')
#####Change the name by your need********
    file = pd.DataFrame({
        "Contig": file["QUERY"],
        "Integron_Type": file["INTEGRON TYPE|ACCESSION"],
        "Start": file["QUERY START"],
        "End": file["QUERY END"]})
    
    file["Location"] = file["Start"].astype(str).str.cat(file["End"].astype(str), sep="-")
    file=file.drop("Start", axis=1)
    file=file.drop("End",axis=1)
#####For Transposon Motifi, code bellow is needed********
    file_contig=[]
    temp_file_contig=file['Contig'].to_list()
    for ii in temp_file_contig:
        ii=ii.split('~')[0]
        file_contig.append(ii)
    file['Contig']=file_contig
#####Change the name by your need********
    file_integron=[]
    temp_file_integron=file['Integron_Type'].to_list()
    for ii in temp_file_integron:
        ii=ii.split('|')[0]
        file_integron.append(ii)
    
    file["Integron_Type"]=file_integron
    file.to_csv(integron_out+i,index=False)

Locate ARG on Transposon

In [ ]:
def search_location_in_file(search_contig, filename, RE_location, data_path):
    """Search for whether RE_location is within the specified range in the given file."""
    global RE_found
    global element_index
    RE_found = 0
    element_index = 'No_Element'
    
    file = pd.read_csv(data_path + filename)
    file = file[file['Contig'] == search_contig]  # Filter rows where the contig matches.
    file_locations = file['Location'].to_list()  # Get the list of values from the location column.
    
    for location in file_locations:
        start, end = map(int, location.split('-'))  # Extract the start and end positions.
        start = max(0, start - 5000)  # Extend 5000 bases upstream.
        end += 5000  # Extend 5000 bases downstream.
        
        RE_start, RE_end = map(int, RE_location.split('-'))  # Obtain the start and end positions of the RE.
        if start <= RE_start and RE_end <= end:
            RE_found = 1  # Find the matching location.
            element_index = file[file['Location'] == location].index[0]  # Obtain the corresponding index.


Function for processing Transposon files.

In [ ]:
def process_transposon_files(transposon_path, ARG_path):
    """Process Transposon files."""
    transposon_list = os.listdir(transposon_path)
    ARG_list = os.listdir(ARG_path)

    Strain, ARG_type, transposon_type = [], [], []

    for transposon_file in transposon_list:
        transposon_name = transposon_file.split('.')[0]
        ARG_file = [x for x in ARG_list if x.startswith(transposon_name)][0]
        
        ARG_data = pd.read_csv(ARG_path + ARG_file, sep='\t')
        contig_list = ARG_data['SEQUENCE'].to_list()
        start_list = ARG_data['START'].to_list()
        end_list = ARG_data['END'].to_list()
        RE_name_list = ARG_data['GENE'].to_list()

        final_count = []
        temp_transposon_type = []

        for idx, contig in enumerate(contig_list):
            RE_location = f"{start_list[idx]}-{end_list[idx]}"
            search_location_in_file(contig, transposon_file, RE_location, transposon_path)
            final_count.append(RE_found)
            temp_transposon_type.append(element_index)

        transposon_type.append(_process_element_type(temp_transposon_type, transposon_file, transposon_path))
        ARG_type.append(_process_ARG_type(final_count, RE_name_list))
        Strain.append(transposon_file.replace('.tsv', ''))

    _save_results(Strain, ARG_type, transposon_type, 'Transposon')


Function for processing Integron files.

In [ ]:
def process_integron_files(integron_path, ARG_path):
    """Process Integron files."""
    integron_list = os.listdir(integron_path)
    ARG_list = os.listdir(ARG_path)

    Strain, ARG_type, integron_type = [], [], []

    for integron_file in integron_list:
        integron_name = integron_file.split('.')[0]
        ARG_file = [x for x in ARG_list if x.startswith(integron_name)][0]
        
        ARG_data = pd.read_csv(ARG_path + ARG_file, sep='\t')
        contig_list = ARG_data['SEQUENCE'].to_list()
        start_list = ARG_data['START'].to_list()
        end_list = ARG_data['END'].to_list()
        RE_name_list = ARG_data['GENE'].to_list()

        final_count = []
        temp_integron_type = []

        for idx, contig in enumerate(contig_list):
            RE_location = f"{start_list[idx]}-{end_list[idx]}"
            search_location_in_file(contig, integron_file, RE_location, integron_path)
            final_count.append(RE_found)
            temp_integron_type.append(element_index)

        integron_type.append(_process_element_type(temp_integron_type, integron_file, integron_path))
        ARG_type.append(_process_ARG_type(final_count, RE_name_list))
        Strain.append(integron_file.replace('.tsv', ''))

    _save_results(Strain, ARG_type, integron_type, 'Integron')


辅助函数

In [ ]:
def _process_element_type(element_type_list, filename, path):
    """Process Transposon/Integron type based on the matching results."""
    if all(item == "No_Element" for item in element_type_list):
        return 'No_Element carrier'

    element_types = []
    numbers_only = [item for item in element_type_list if item != "No_Element"]
    element_file = pd.read_csv(path + filename)
    element_info = element_file.iloc[:, -2].to_list()  # Assume that the type information is in the second-to-last column.
    for index in numbers_only:
        element_types.append(element_info[index])

    return element_types

def _process_ARG_type(final_count, RE_name_list):
    """Process ARG type."""
    if 1 in final_count:
        ARG_type_list = [RE_name_list[idx] for idx, count in enumerate(final_count) if count == 1]
        return ARG_type_list
    else:
        return "No_Element-borne ARG"

def _save_results(Strain, ARG_type, element_type, element_name):
    """Save the results to a CSV file."""
    output_dir = 'Path/to/'
    output_filename = f'result_{element_name}.csv'
    output_path = output_dir + output_filename

    data = {
        'Strain': Strain,
        'ARG_type': ARG_type,
        f'{element_name}_type': element_type
    }
    
    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False)


主函数

In [ ]:
if __name__ == "__main__":
    transposon_path = 'Path/to/Motifi_transposon/'
    integron_path = 'Path/to/Motifi_integron/'
    ARG_path = 'Path/to/test_abricate_result/'

    print("Processing Transposon files...")
    process_transposon_files(transposon_path, ARG_path)

    print("Processing Integron files...")
    process_integron_files(integron_path, ARG_path)
